In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [4]:
dataset = pd.read_csv('./data/dataset_cal.csv')

remove = ["Subject", "Activity"]
features = [column for column in list(data.columns) if column not in remove]
dataset['Gender'].replace(' f', 0, inplace=True)
dataset['Gender'].replace(' m', 1, inplace=True)
dataset.drop(columns = ['WEIGHT','Gender', 'AGE', 'HEIGHT', 'SKIN', 'SPORT', 'Activity','Subject' ], inplace = True)

In [6]:

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
tf.random.set_seed(7)
# load the dataset
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean absolute error
trainScore = np.sqrt(mean_absolute_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_absolute_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

Epoch 1/100
43342/43342 - 31s - loss: 6.8451e-04 - 31s/epoch - 706us/step
Epoch 2/100
43342/43342 - 30s - loss: 1.7424e-04 - 30s/epoch - 703us/step
Epoch 3/100
43342/43342 - 32s - loss: 1.7111e-04 - 32s/epoch - 730us/step
Epoch 4/100
43342/43342 - 31s - loss: 1.6938e-04 - 31s/epoch - 726us/step
Epoch 5/100
43342/43342 - 31s - loss: 1.6822e-04 - 31s/epoch - 712us/step
Epoch 6/100
43342/43342 - 31s - loss: 1.6720e-04 - 31s/epoch - 711us/step
Epoch 7/100
43342/43342 - 31s - loss: 1.6641e-04 - 31s/epoch - 705us/step
Epoch 8/100
43342/43342 - 31s - loss: 1.6639e-04 - 31s/epoch - 714us/step
Epoch 9/100
43342/43342 - 31s - loss: 1.6583e-04 - 31s/epoch - 706us/step
Epoch 10/100
43342/43342 - 31s - loss: 1.6573e-04 - 31s/epoch - 724us/step
Epoch 11/100
43342/43342 - 31s - loss: 1.6556e-04 - 31s/epoch - 704us/step
Epoch 12/100
43342/43342 - 32s - loss: 1.6525e-04 - 32s/epoch - 730us/step
Epoch 13/100
43342/43342 - 31s - loss: 1.6499e-04 - 31s/epoch - 714us/step
Epoch 14/100
43342/43342 - 31s - l

ValueError: non-broadcastable output operand with shape (43342,1) doesn't match the broadcast shape (43342,35)